In [1]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten

from sklearn.preprocessing import OneHotEncoder
import gensim 
from gensim.models import Word2Vec 

Using TensorFlow backend.


In [2]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [3]:
# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [4]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [5]:
print(doc)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

## embedding style

#### one hot encoding

In [7]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

### Word Vector

#### Glove encoding

In [8]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(sequences, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B/glove.6B.100d.txt', encoding='utf8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
Glove_model = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
# evaluate the model
# loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

[[   30  1846    13  3276]
 [ 1846    13  3276    35]
 [   13  3276    35  3887]
 ...
 [ 3886 10649     4   320]
 [10649     4   320    85]
 [    4   320    85   314]]
Loaded 400000 word vectors.


#### CBOW embedding

In [9]:
model_cbow = gensim.models.Word2Vec(tokenizer.word_index, min_count = 1,  
                              size = 100, window = 5) 

In [19]:
print(model_cbow.syn1neg)

[[-0.06645159 -0.20713907  0.18939526 ...  0.01618759 -0.0093843
   0.04583556]
 [-0.06661418 -0.20868962  0.16612875 ...  0.00054999 -0.02618063
   0.05085934]
 [-0.03335868 -0.2057877   0.1834178  ...  0.03435163 -0.07040621
   0.01445945]
 ...
 [-0.01075645 -0.06878496  0.06222077 ...  0.00806859 -0.02605717
   0.00285659]
 [-0.0135956  -0.07445689  0.05915261 ...  0.00907374 -0.02075437
   0.00936771]
 [-0.02035927 -0.09541447  0.08315407 ...  0.00941986 -0.02595772
   0.01146735]]


<ipython-input-19-f3da59bb6fa3>:1: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  print(model_cbow.syn1neg)


In [26]:
# define model
model = Sequential()
# keras embedding
model.add(Embedding(vocab_size, 8, weight= model_cbow.syn1neg, input_length=seq_length))

# cbow embedding
# model.add(Glove_model)

model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

<ipython-input-26-c9238d0b6691>:4: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  model.add(Embedding(vocab_size, 8, weight= model_cbow.syn1neg, input_length=seq_length))


TypeError: ('Keyword argument not understood:', 'weight')

In [21]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            1065000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 100)            80400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10650)             1075650   
Total params: 2,311,550
Trainable params: 1,246,550
Non-trainable params: 1,065,000
_________________________________________________________________
None


In [22]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
# fit model
model.fit(X, y, batch_size=128, epochs=100)
# fit model glove
# model.fit(padded_docs, max_length, batch_size=128, epochs=100)

ValueError: Error when checking input: expected embedding_1_input to have shape (4,) but got array with shape (50,)

In [ ]:
# save the model to file
model.save('model.h5')

In [ ]:
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))